In [ ]:
# -*- coding: utf-8 -*-

import openpyxl
import numpy as np
import matplotlib.pyplot as plt
import keras

TD = openpyxl.load_workbook('totaldata_mice_v1.xlsx', data_only=True)
Sheet = TD['totaldata_mice']

input_dim = 2
total_length = 486
x = np.zeros((total_length,input_dim))
for i in range(total_length):
    x[i,0] = Sheet.cell(row = 2+i, column = 2).value
    x[i,1] = Sheet.cell(row = 2+i, column = 4).value

def Normalizing(A):
    denom = np.max(np.abs(A)) + 1e-6
    return A/denom, denom

output_seq_len = 5

#test_rate = 0.2
train_rate = 0.5
valid_rate = 0.3

diff_x = np.zeros((total_length-1,input_dim))
for j in range(total_length-1):
    diff_x[j,:] = (x[j+1,:] - x[j,:])/x[j,:]

input_seq_len = 20

_, denom_1 = Normalizing(diff_x[:,0])
_, denom_2 = Normalizing(diff_x[:,1])

norm_diff_x = np.zeros(diff_x.shape)
norm_diff_x[:,0] = diff_x[:,0]/denom_1
norm_diff_x[:,1] = diff_x[:,1]/denom_2

num_data = total_length - input_seq_len - output_seq_len

train_size = int(train_rate*num_data)
valid_size = int(valid_rate*num_data)
test_size = num_data - train_size - valid_size

output_dim = 1

data_x = np.zeros((num_data,input_seq_len,input_dim))
data_y = np.zeros((num_data,output_seq_len,output_dim))

for da in range(num_data):
    data_x[da,:,:] = norm_diff_x[da:da+input_seq_len,:]
    data_y[da,:,:] = norm_diff_x[da+input_seq_len:da+input_seq_len+output_seq_len,0].reshape(output_seq_len,output_dim)

original_data_x = np.zeros(test_size)
original_data_y = np.zeros((test_size,output_seq_len))

for ot in range(test_size):
    original_data_x[ot] = x[input_seq_len+train_size+valid_size+ot,0]
    original_data_y[ot,:] = x[input_seq_len+train_size+valid_size+ot+1:input_seq_len+train_size+valid_size+ot+1+output_seq_len,0]

shuffled_index = np.random.permutation(train_size+valid_size)
test_x = data_x[train_size+valid_size:,:,:]
test_y = data_y[train_size+valid_size:,:,:]
train_x = data_x[shuffled_index[:train_size],:,:]
train_y = data_y[shuffled_index[:train_size],:,:]
valid_x = data_x[shuffled_index[train_size:train_size+valid_size],:,:]
valid_y = data_y[shuffled_index[train_size:train_size+valid_size],:,:]

hidden_size = 128
patience = 50

model = keras.Sequential()
model.add(keras.layers.LSTM(hidden_size, input_shape = (input_seq_len, input_dim), return_sequences = True))
model.add(keras.layers.LSTM(hidden_size, return_sequences = True))
model.add(keras.layers.LSTM(hidden_size))
model.add(keras.layers.Dense(hidden_size, activation ='relu'))
model.add(keras.layers.RepeatVector(output_seq_len))
model.add(keras.layers.LSTM(hidden_size, return_sequences = True))
model.add(keras.layers.LSTM(hidden_size, return_sequences = True))
model.add(keras.layers.LSTM(hidden_size, return_sequences = True))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(output_dim)))
model.compile(loss='mean_squared_error' , optimizer= 'adam')
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience),
            keras.callbacks.ModelCheckpoint(filepath='yjlee_model.h5', monitor='val_loss', save_best_only=True)]
model.fit(train_x, train_y, epochs = 1000, callbacks = callbacks, validation_data=(valid_x, valid_y), verbose = 1)
model = keras.models.load_model('yjlee_model3.h5')
loss = model.evaluate(valid_x, valid_y, verbose = 0)
print(loss)

predicted = model.predict(test_x, verbose = 0)
predicted_y = np.zeros((test_size,output_seq_len))
predicted_y[:,0] = original_data_x*(1+predicted[:,0,0]*denom_1)
for os in range(1,output_seq_len):
    predicted_y[:,os] = predicted_y[:,os-1]*(1+predicted[:,os,0]*denom_1)

error = np.sqrt(np.mean(np.square(predicted_y - original_data_y)))
print(error)
print(predicted_y[-1,:],original_data_y[-1,:])

    
    
    
